## Starter notebook for single-cell nuclear RNA analysis
### from scanpy tutorials

In [1]:
import sys
print(sys.executable)
### load packages
import anndata as ad # for annotating with obs and var
import scanpy as sc # core sc package
import pandas as pd # dataframes
#import harmonypy as hm # for batch effect correction
#import scanorama # for batch effect correction 
import scvi as scvi # for doublet handling
import seaborn as sns # plots
import matplotlib.pyplot as plt # plots
import scipy.sparse as sp
from scipy.sparse import csr_matrix # matrices
import sys # system
import os # system
import glob # global system
from os import listdir
from os.path import isfile, join
#import torch # for training scvi
import random
import numpy as np # numbers
from scipy.stats import median_abs_deviation as mad
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

/software/cellgen/team274/lr26/miniforge3/envs/my-python/bin/python


/software/cellgen/team274/lr26/miniforge3/envs/my-python/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/software/cellgen/team274/lr26/miniforge3/envs/my-python/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [2]:
np.random.seed(42)  # Set seed for 

In [ ]:
# understanding the cellranger output - it looks for perfect transcript matches so introns not really relevant
# Use deep learning (scVI) for doublet removal
# Account for nuclear-specific QC metrics (lower gene counts than scRNA-seq)
# Be cautious with mitochondrial genes (not always high in nuclei)
# batch correction or multiple sample integration

In [3]:
# set base directory
base_dir = "/lustre/scratch126/cellgen/behjati/lr26/snRNA/"

In [4]:
# Find all folders ending with '_filtered_feature_bc_matrix'
matrix_dir_list = glob.glob(os.path.join(base_dir, "*_filtered_feature_bc_matrix"))

def get_sample_id(matrix_dir_path):
    foldername = os.path.basename(matrix_dir_path)
    # Assumes sample ID is first three parts separated by "_", e.g. CG_SB_NB13960948
    sample_id = "_".join(foldername.split("_")[0:3])
    return sample_id

In [5]:
def run_scrublet(adata, expected_rate=0.1, n_pcs=10):
    try:
        # Filter out cells with zero counts
        counts_per_cell = np.array(adata.X.sum(axis=1)).flatten()
        adata = adata[counts_per_cell > 0].copy()

        sc.pp.scrublet(
            adata,
            expected_doublet_rate=expected_rate,
            threshold=None,  # auto-estimate threshold
            n_prin_comps=n_pcs,
            verbose=True
        )

        adata.obs["scrublet"] = adata.obs["predicted_doublets"].map({True: "doublet", False: "singlet"})
        adata.obs.rename(columns={"doublet_score": "scrublet_score"}, inplace=True)

    except Exception as e:
        print(f"Scrublet failed: {e}")

    return adata

In [6]:
def load_and_merge_data(matrix_dir_list):
    adatas = []

    for matrix_dir in matrix_dir_list:
        sample_id = get_sample_id(matrix_dir)

        # Load the 10X MTX matrix from folder
        adata = sc.read_10x_mtx(
            matrix_dir,
            var_names="gene_symbols",  # or "gene_ids"
            cache=True
        )
        adata.var_names_make_unique()

        print(f"Loaded {sample_id}: {adata.shape}")

        # Run Scrublet
        adata = run_scrublet(adata)

        # Annotate metadata
        adata.obs["CellID"] = [sample_id + ":" + bc for bc in adata.obs_names]
        adata.obs["SampleID"] = sample_id
        adata.obs["Barcode"] = adata.obs_names

        adatas.append(adata)

    # Merge all samples into one AnnData object
    adata_merged = adatas[0].concatenate(*adatas[1:], index_unique="-")

    print(f"Merged data shape: {adata_merged.shape}")

    return adata_merged

# Run the whole pipeline
adata = load_and_merge_data(matrix_dir_list)

Loaded CG_SB_NB13960949: (43714, 33694)
Scrublet failed: 'predicted_doublets'
Loaded CG_SB_NB14449540: (13184, 33694)
Scrublet failed: 'predicted_doublets'
Loaded CG_SB_NB13960950: (1566, 33694)
Scrublet failed: 'predicted_doublets'
Loaded CG_SB_NB13960948: (2084, 33694)
Scrublet failed: 'predicted_doublets'
Loaded CG_SB_NB14449539: (12163, 33694)
Scrublet failed: 'predicted_doublets'
Loaded CG_SB_NB14449541: (11743, 33694)
Scrublet failed: 'predicted_doublets'
Loaded CG_SB_NB13960951: (1075, 33694)
Scrublet failed: 'predicted_doublets'


/tmp/ipykernel_1852003/1329535910.py:28: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_merged = adatas[0].concatenate(*adatas[1:], index_unique="-")


Merged data shape: (85529, 33694)


In [ ]:
# Find all directories ending with "_filtered_feature_bc_matrix"
matrix_dirs = glob.glob(os.path.join(base_dir, "*_filtered_feature_bc_matrix"))

# Function to extract sample ID from file name
def get_sample_id(matrix_dir_path):
    foldername = os.path.basename(matrix_dir_path)
    sample_id = "_".join(foldername.split("_")[0:3])  # CG_SB_NBxxxxxxx
    return sample_id

# Example: get sample IDs for all files
for matrix_dir in matrix_dirs:
    sample_id = get_sample_id(matrix_dir)
    print(f"{sample_id}: {matrix_dir}")


In [7]:
adata

AnnData object with n_obs × n_vars = 85529 × 33694
    obs: 'doublet_score', 'predicted_doublet', 'CellID', 'SampleID', 'Barcode', 'batch'
    var: 'gene_ids', 'feature_types'

In [ ]:
# quality control calculations
# Calculate QC metricsm - do this first
# number of genes detected per cell
# Total UMIs per cell
# Mitochondrial gene percentage
# Ribosomal gene percentage

adata_merged.var["mt_gene"] = adata_merged.var_names.str.startswith(("MT-", "mt-"))  # mitochondrial genes, handles anything starting with mt
adata_merged.var["ribo_gene"] = adata_merged.var_names.str.contains(("^RP[SL][0-9]"), regex=True)  # ribosomal genes anything that contains the regex
#calculate quality metrics
sc.pp.calculate_qc_metrics(adata_merged, qc_vars=["mt_gene", "ribo_gene"], percent_top=None, log1p=False, inplace=True)

In [ ]:
# visualisation of the counts with violin plot
sc.pl.violin(adata_merged, ["n_genes_by_counts", "total_counts"], jitter=0.4)
sc.pl.violin(adata_merged, ["pct_counts_mt_gene", "pct_counts_ribo_gene"], jitter=0.4)
sc.pl.scatter(adata_merged, x="total_counts", y="n_genes_by_counts")

In [ ]:
print(adata_merged) # how does it look
print(adata_merged.obs.head()) #cells
print(adata_merged.var.head())

In [ ]:
print(adata_merged) # how does it look
print(adata_merged.obs.head()) #cells
print(adata_merged.var.head())
print(adata_merged.shape)  # (n_cells, n_genes)
type(adata_merged.X) # what type of data is it

In [ ]:
#adata_x.X[1:20,15:40].todense()
print(adata_merged.X[1:20, 15:40].toarray())  # safer for memory to have a look at the counts

In [ ]:
# Step 1: Run PCA (if you haven't already, as UMAP relies on PCA or other dimensionality reduction)
sc.tl.pca(adata_merged, svd_solver='arpack')

# Step 2: Compute the nearest neighbors graph
sc.pp.neighbors(adata_merged, n_neighbors=15, n_pcs=50)  # n_neighbors can be adjusted

# Step 3: Calculate the UMAP embeddings
sc.tl.umap(adata_merged)

# Step 4: Now you can visualize the UMAP plot with 'scrublet' and 'scrublet_score' colorings
sc.pl.umap(adata_merged, color=["scrublet", "scrublet_score"])

#Step 2: Visualize doublets vs singlets based on library size (total counts)
sc.pl.scatter(adata_merged, x="total_counts", y="n_genes_by_counts", color="scrublet", title="Doublets vs Singlets based on Library Size", palette=["grey", "red"])

# You can also use a UMAP plot if you have UMAP coordinates
sc.pl.umap(adata_merged, color="scrublet", title="Doublets vs Singlets on UMAP")

In [ ]:
sns.histplot(adata_merged.obs["scrublet_score"], bins=50, kde=True)

In [ ]:
# Count occurrences of each category in the 'scrublet' column
doublet_counts = adata_merged.obs["scrublet"].value_counts()
# Print the count of doublets
print(f"Number of doublets identified: {doublet_counts['doublet']}")

In [ ]:
adata_previous = sc.read_h5ad(base_dir + "adata_merged_after_scrumblet.h5ad")

In [ ]:
adata_merged.write(base_dir + "adata_merged_new_env_scrublet.h5ad")

In [ ]:
adata_new = sc.read_h5ad(base_dir + "adata_merged_after_scrumblet_scvi.h5ad")

In [ ]:
print(adata_scrscvi) # how does it look
print(adata_scrscvi.obs.head()) #cells
print(adata_scrscvi.var.head())
print(adata_scrscvi.shape)  # (n_cells, n_genes)
type(adata_scrscvi.X) # what type of data is it

In [ ]:
print(adata_previous.obs.index.duplicated().sum())  # Should be 0
print(adata_new.obs.index.duplicated().sum()) 

In [ ]:
# Count occurrences of each category in the 'scrublet' column
doublet_counts = adata_previous.obs["scrublet"].value_counts()
# Print the count of doublets
print(f"Number of doublets identified: {doublet_counts['doublet']}")

In [ ]:
# convert sparse matrix to a dense for checking
X_dense = adata_merged.X.toarray() if sp.issparse(adata_merged.X) else adata_merged.X
# NaN values - none there
print("NaN values in expression matrix:", np.isnan(X_dense).sum())
# infinite values - none there
print("Infinite values in expression matrix:", np.isinf(X_dense).sum())

In [ ]:
adata_new.layers["counts"] = adata_new.X.copy()
#adata_previous.layers["counts"] = adata_previous.X.copy()

In [ ]:
# Use all CPU cores for PyTorch
torch.set_num_threads(1)

def run_scvi_solo(adata, n_epochs=50, use_gpu=False):
    """Runs SCVI and SOLO on a given AnnData object."""
    
    # Create a copy of the dataset to avoid modifying the original
    adata_copy = adata.copy()
    
    # SCVI setup for AnnData
    scvi.model.SCVI.setup_anndata(
        adata_copy, 
        batch_key="SampleID",  # Specify batch correction key
        layer="counts"  # Raw counts should be stored in the 'counts' layer
    )
    
    # Train SCVI model
    vae = scvi.model.SCVI(adata_copy)
    vae.train(max_epochs=n_epochs)

    # Run SOLO for doublet detection
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train(max_epochs=n_epochs)

    # Get SOLO predictions
    df = solo.predict()
    df["prediction"] = solo.predict(soft=False)
    df["dif"] = df.doublet - df.singlet  # Confidence score
    df.index = df.index.map(lambda x: x[:-2])  # Adjust cell names if needed

    # Store results in the AnnData object
    adata_copy.obs["scVI_SOLO"] = df["prediction"]
    adata_copy.obs["scVI_SOLO_score"] = df["dif"]
    
    return adata_copy

In [ ]:
def run_scvi_solo(adata, n_epochs=50, use_gpu=False):
    """Runs SCVI and SOLO on a given AnnData object."""
    
    # Create a copy of the dataset
    adata_copy = adata.copy()
    
    # Ensure unique cell names
    if adata_copy.obs.index.duplicated().sum() > 0:
        print("⚠️ Warning: Duplicate cell names detected. Fixing them...")
        adata_copy.obs.index = adata_copy.obs.index + "_" + adata_copy.obs.groupby(level=0).cumcount().astype(str)

    # SCVI setup for AnnData
    scvi.model.SCVI.setup_anndata(
        adata_copy, 
        batch_key="SampleID",
        layer="counts"
    )
    
    # Train SCVI
    vae = scvi.model.SCVI(adata_copy)
    vae.train(max_epochs=n_epochs)

    # Train SOLO
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train(max_epochs=n_epochs)

    # Get predictions
    df = solo.predict()
    df["prediction"] = solo.predict(soft=False)
    df["dif"] = df.doublet - df.singlet

    # Ensure SOLO's cell names match AnnData
    df.index = df.index.str.replace("_1$", "", regex=True)  # Adjust if needed

    # Match indices before assignment
    df = df.loc[df.index.intersection(adata_copy.obs.index)]  # Keep only matching cells

    # Store results
    adata_copy.obs.loc[df.index, "scVI_SOLO"] = df["prediction"]
    adata_copy.obs.loc[df.index, "scVI_SOLO_score"] = df["dif"]
    
    return adata_copy

### Run scVI-SOLO on both datasets
solo_results_new = run_scvi_solo(adata_new, n_epochs=50)

# Append results to the original datasets
adata_new.obs["scVI_SOLO"] = solo_results_new.obs["scVI_SOLO"]
adata_new.obs["scVI_SOLO_score"] = solo_results_new.obs["scVI_SOLO_score"]

adata_new.write(base_dir + "adata_merged_new_after_scrumblet_scvi.h5ad")

In [ ]:
### Run scVI-SOLO on both datasets
solo_results_previous = run_scvi_solo(adata_previous, n_epochs=50)

# Append results to the original datasets
adata_previous.obs["scVI_SOLO"] = solo_results_previous.obs["scVI_SOLO"]
adata_previous.obs["scVI_SOLO_score"] = solo_results_previous.obs["scVI_SOLO_score"]

adata_previous.write(base_dir + "adata_merged_after_scrumblet_scvi.h5ad")

In [ ]:
### Run scVI-SOLO on both datasets
solo_results_new = run_scvi_solo(adata_new, n_epochs=50)

# Append results to the original datasets
adata_new.obs["scVI_SOLO"] = solo_results_new.obs["scVI_SOLO"]
adata_new.obs["scVI_SOLO_score"] = solo_results_new.obs["scVI_SOLO_score"]

adata_new.write(base_dir + "adata_merged_new_after_scrumblet_scvi.h5ad")

In [ ]:
adata_scvi = sc.read_h5ad(base_dir + "adata_merged_after_scrumblet_scvi.h5ad")

In [ ]:
print(adata_scvi) # how does it look
print(adata_scvi.obs.head()) #cells
print(adata_scvi.var.head())
print(adata_scvi.shape)  # (n_cells, n_genes)
type(adata_scvi.X) # what type of data is it

In [ ]:
solo_results_merged = run_scvi_solo(adata_merged, n_epochs=50)
adata_merged.obs["scVI_SOLO"] = solo_results_merged.obs["scVI_SOLO"]
adata_merged.obs["scVI_SOLO_score"] = solo_results_merged.obs["scVI_SOLO_score"]

In [ ]:
# Append results to the original datasets
adata_previous.obs["scVI_SOLO"] = solo_results_previous.obs["scVI_SOLO"]
adata_previous.obs["scVI_SOLO_score"] = solo_results_previous.obs["scVI_SOLO_score"]

adata_merged.obs["scVI_SOLO"] = solo_results_merged.obs["scVI_SOLO"]
adata_merged.obs["scVI_SOLO_score"] = solo_results_merged.obs["scVI_SOLO_score"]

In [ ]:
print(adata_merged.X.shape)  # Check the shape of the main data matrix


In [ ]:
sc.pp.normalize_total(adata_merged, target_sum=1e4)  # normalize per cell
sc.pp.log1p(adata_merged)  # Log transform

In [ ]:
print(adata_x.obs.groupby("scVI_SOLO").size())
print(adata_x.obs.groupby("scrublet").size())


In [ ]:

# 🔹 Filter out doublets from `adata_merged`
adata_filtered = adata_merged[adata_merged.obs["scVI_SOLO"] == "singlet", :]

# ✅ Save the filtered dataset
adata_filtered.write(base_dir + "adata_filtered.h5ad")

# 🔹 Check how many cells were removed
print(f"Original: {adata_merged.shape}, Filtered: {adata_filtered.shape}")

In [ ]:
# (7) Extract latent embeddings
adata_filtered.obsm["X_scVI"] = model.get_latent_representation()

# (8) Run UMAP/clustering
sc.pp.neighbors(adata_filtered, use_rep="X_scVI")
sc.tl.umap(adata_filtered)
sc.tl.leiden(adata_filtered, resolution=0.5)

# (9) Visualize
sc.pl.umap(adata_filtered, color=["SampleID", "leiden", "scrublet"])


In [ ]:
# Merge all datasets
adata_merged = adatas[0].concatenate(*adatas[1:], batch_key="SampleID", index_unique="-")

# Check for duplicates based on barcodes
# Assuming barcodes are stored in the obs_names column or an explicit 'Barcode' column
# Remove duplicates based on barcode (if necessary)
adata_merged = adata_merged[~adata_merged.obs_names.duplicated(), :]

# Alternatively, you can remove duplicates based on CellID if that's a more appropriate column
adata_merged = adata_merged[~adata_merged.obs["CellID"].duplicated(), :]


In [ ]:
#filter dying cells
adata = adata[adata.obs.n_genes_by_counts > 200, :]  # Remove nuclei with very few genes
adata = adata[adata.obs.total_counts < 100000, :]  # Remove outliers
adata = adata[adata.obs.pct_counts_mt < 5, :]  # Remove cells with >5% mito reads
adata = adata[adata.obs.pct_counts_ribo < 10, :]  # Remove high ribosomal content

# Set thresholds (adjust based on dataset)
min_genes = 200  # Keep cells with at least 200 genes
max_mito = 5     # Remove cells with >5% mitochondrial content

# Filter low-quality cells
adata = adata[(adata.obs.n_genes_by_counts > min_genes) & 
              (adata.obs.pct_counts_mt < max_mito)]


In [ ]:
# normalisation of the data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
adata = adata[:, adata.var.highly_variable]  # Keep only highly variable genes
sc.pp.scale(adata, max_value=10)


In [ ]:
# correction for batch effects
# Compute PCA
sc.tl.pca(adata, svd_solver="arpack")
# Run Harmony batch correction
harmony = hm.run_harmony(adata.obsm["X_pca"], adata.obs, "batch")
adata.obsm["X_pca_harmony"] = harmony.Z_corr.T  # Use corrected PCA embeddings
# Recompute neighbors and UMAP on corrected embeddings
sc.pp.neighbors(adata, use_rep="X_pca_harmony")
sc.tl.umap(adata)
# Visualize batch correction effect
sc.pl.umap(adata, color=["batch", "cell_type"])

import scanorama
# Split dataset by batch
batches = [adata[adata.obs.batch == b] for b in adata.obs.batch.unique()]
# Apply Scanorama integration
integrated_batches, corrected_matrices = scanorama.correct_scanpy(batches)
# Merge back into one AnnData object
adata_corrected = sc.AnnData.concatenate(*integrated_batches, batch_key="batch")
# Run PCA & clustering
sc.tl.pca(adata_corrected)
sc.pp.neighbors(adata_corrected)
sc.tl.umap(adata_corrected)
# Visualize
sc.pl.umap(adata_corrected, color=["batch", "cell_type"])

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="cell_type", method="wilcoxon")
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)

df = sc.get.rank_genes_groups_df(adata, group="Neurons")
df.to_csv("DEGs_Neurons.csv")


In [ ]:
adata.write("snRNA_final.h5ad")
sc.pl.umap(adata, color=["cell_type", "n_genes_by_counts"])


In [ ]:
### Holly code
def run_scrublet(adata_x):
    
    scrub = scr.Scrublet(adata_x.X.todense())
    
    try:
        doublet_scores, predicted_doublets = scrub.scrub_doublets()
        adata_x.obs["scrublet"] = ["doublet" if i == True else "singlet" for i in predicted_doublets]
        adata_x.obs["scrublet_score"] = doublet_scores
        return adata_x
    except:
        print("Scrublet failed")
        return adata_x

    ##### ----- Add scrublet per patient
    adata_x = run_scrublet(adata_x)
    qc_data_x  = adata_x.obs

    ## -- Filter cells
    print(sample_x)
    adata_x.var["mito"] = adata_x.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(adata_x, qc_vars=["mito"], inplace=True)
    gene_thresh = 500
    mito_thresh = 15
    to_discard = np.array(qc_data_x["pct_counts_mito"]>mito_thresh) | np.array(qc_data_x["n_genes_by_counts"]<float(gene_thresh))
    cells_to_keep = list(qc_data_x.loc[~to_discard]["CellID"])
    
    print("Before: "+str(adata_x.shape[0])+" cells")
    adata_x = adata_x[adata_x.obs_names.isin(cells_to_keep)]
    print("After: "+str(adata_x.shape[0])+" cells")
    
    return adata_x

adat_lst = [loadAnnData(i) for i in folders_to_load]
adata = ad.concat(adat_lst,join="outer", label=None)



In [ ]:
adata = sc.read_h5ad(base_dir + "adata_merged_scrublet_scvi_umi_cons_doub.h5ad")

In [ ]:
adata

In [ ]:
# MAD outlier detection
def mad_outlier(adata, metric, nmads):
    M = adata.obs[metric]
    return (M < np.median(M) - nmads * mad(M)) | (M > np.median(M) + nmads * mad(M))

In [ ]:
# Get Soupx groups
def get_soupx_group(adata):
    adata_pp = adata.copy()
    sc.pp.normalize_per_cell(adata_pp)
    sc.pp.log1p(adata_pp)
    sc.pp.pca(adata_pp)
    sc.pp.neighbors(adata_pp)
    sc.tl.leiden(adata_pp, key_added="soupx_groups")
    return adata_pp.obs['soupx_groups']

In [ ]:
# Prepare broth data for SoupX
def prepare_broth(adata):
    cells = adata.obs_names
    genes = adata.var_names
    data = adata.X.T
    
    # Get raw data (assuming it's stored separately in your case)
    sample_id = adata.obs.iloc[0]['SampleID']
    raw = sc.read_10x_mtx("/lustre/scratch126/casm/team274sb/lr26/scRNA/" + sample_id + "/raw_feature_bc_matrix/").X.T
    
    # Get leiden clusters
    soupx_groups = get_soupx_group(adata)

    return data, raw, genes, cells, soupx_groups

In [ ]:
# Function to run SoupX analysis in R
import rpy2.robjects as r

# Define the R function (inside Python)
r.r("""
    library(SoupX)

    make_soup <- function(data, raw, genes, cells, soupx_groups){
        rownames(data) <- genes
        colnames(data) <- cells
        data <- as(data, "sparseMatrix")
        raw <- as(raw, "sparseMatrix")
        
        sc <- SoupChannel(raw, data, calcSoupProfile = FALSE)
        
        soupProf <- data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
        sc <- setSoupProfile(sc, soupProf)
        
        sc <- setClusters(sc, soupx_groups)
        
        sc <- autoEstCont(sc, tfidfMin = 0.1, soupQuantile = 0.1, doPlot=FALSE, forceAccept=TRUE)
        out <- adjustCounts(sc, roundToInt = TRUE)

        return(out)
    }
""")




In [ ]:
# Function to cook soup and add layers
def cook_soup(adata):
    data, raw, genes, cells, soupx_groups = prepare_broth(adata)

    # Execute the R code and get the corrected counts
    %R -i data -i raw -i genes -i cells -i soupx_groups -o out out = make_soup(data, raw, genes, cells, soupx_groups)

    # Store raw counts and SoupX counts
    adata.layers["raw_counts"] = adata.X
    adata.layers["soupX_counts"] = out.T
    adata.X = adata.layers["soupX_counts"]
    
    return adata

In [ ]:
# Example: Process your large AnnData object
def process_samples(adata):
    obsm_global = adata.obsm.copy()  # Keep original global 'obsm' (embeddings like PCA, UMAP)
    varm_global = adata.varm.copy()  # Keep original global 'varm' (e.g., PCA loadings)
    uns_global = adata.uns.copy()    # Keep original global 'uns' (metadata, neighbors, etc.)
    var_global = adata.var.copy() 
    obsp_global = adata.obsp.copy() # Keep the 'var' data for genes
    combined_adata = []
    sample_ids = adata.obs['SampleID'].unique()

    for sample_id in sample_ids:
        # Extract individual sample's AnnData object
        adata_sample = adata[adata.obs['SampleID'] == sample_id].copy()
        adata_sample.var = adata.var.copy()  # Ensure 'var' is kept as-is
        adata_sample.uns = adata.uns.copy()  # Copy the 'uns' metadata
        
        # Run your analysis (e.g., SoupX or other preprocessing here)
        adata_sample = cook_soup(adata_sample)  # Assuming cook_soup is defined elsewhere
    
        # Append processed data to the combined list
        combined_adata.append(adata_sample)

    # Combine all individual AnnData objects back into one
    adata_combined = ad.concat(combined_adata, join='outer', label='SampleID')
    adata_combined.obsm = obsm_global
    adata_combined.varm = varm_global
    adata_combined.uns = uns_global
    adata_combined.var = var_global
    adata_combined.obsp = obsp_global
    # Optionally, if you need to add or modify attributes after concatenation, you can do so here
    # For example:
    # adata_combined.uns['some_key'] = some_value

    return adata_combined




In [ ]:
# Example: Process your large AnnData object
adata_combined = process_samples(adata)

In [ ]:
adata_combined.write(base_dir + "adata_merged_scrublet_scvi_umi_cons_doub_soupx.h5ad")

In [ ]:
adata_combined